In [30]:
import pandas as pd
from gurobipy import Model, GRB, quicksum

# Load and clean your data
data = pd.read_csv('/Users/cindychang/Documents/school/大二/OR/midterm/data/instance05.csv')

def parse_machine_list(machine_list):
    if pd.isna(machine_list):
        return []
    return list(map(int, machine_list.split(',')))

data['Stage-1 Machines'] = data['Stage-1 Machines'].apply(parse_machine_list)
data['Stage-2 Machines'] = data['Stage-2 Machines'].apply(parse_machine_list)

# set up all the machine's id
all_machines = set()
data['Stage-1 Machines'].apply(lambda machines: all_machines.update(machines))
data['Stage-2 Machines'].apply(lambda machines: all_machines.update(machines))


# Initialize the model
model = Model("Job_Scheduling")

# Extract jobs, machines, and assume stages and data are set
jobs = data['Job ID'].unique()
machines = set(sum(data['Stage-1 Machines'].tolist(), []) + sum(data['Stage-2 Machines'].tolist(), []))

# Define a large M
M = 10000  # Adjust based on your expected schedule bounds

# Add decision variables
s = model.addVars(jobs, [1, 2], name="s", vtype=GRB.CONTINUOUS, lb = 0.0)  # Start times
c = model.addVars(jobs, [1, 2], name="c", vtype=GRB.CONTINUOUS, lb = 0.0)  # Completion times
x = model.addVars(jobs, [1, 2], machines, name="x", vtype=GRB.BINARY)  # Machine assignment
T = model.addVars(jobs, name="T", vtype=GRB.CONTINUOUS, lb = 0.0)  # Tardiness
seq = model.addVars(jobs, jobs, [1, 2], vtype=GRB.BINARY, name="seq")  # Sequencing binary variables


M = 10000

model.setObjective(sum((T[j] for j in jobs), GRB.MINIMIZE))

# Constraints

# # 1. Job assignment for each stage
# for j in jobs:
#     # For stage 1:
#     stage_1_machines = data.at[data.index[data['Job ID'] == j][0], 'Stage-1 Machines']
#     model.addConstr(quicksum(x[j, 1, k] for k in machines if k in stage_1_machines) == 1, name=f"Job_Assignment_Stage1_{j}")
    
#     # For stage 2:
#     stage_2_machines = data.at[data.index[data['Job ID'] == j][0], 'Stage-2 Machines']
#     model.addConstr(quicksum(x[j, 2, k] for k in machines if k in stage_2_machines) == 1, name=f"Job_Assignment_Stage2_{j}")


# 2. Processing time constraint
for j in jobs:
    p1 = data.loc[data['Job ID'] == j, 'Stage-1 Processing Time'].values[0]
    p2 = data.loc[data['Job ID'] == j, 'Stage-2 Processing Time'].values[0]
    model.addConstr(c[j, 1] == s[j, 1] + p1)
    model.addConstr(c[j, 2] == s[j, 2] + p2)

# 3. Stage order constraint
for j in jobs:
    model.addConstr(s[j, 2] >= c[j, 1])

# 4. Machine non-overlap for each stage

for i in jobs:
    for j in jobs:
        if i != j:
            for stage in [1, 2]:
                model.addConstr(s[i, stage] + data.loc[data['Job ID'] == i, f'Stage-{stage} Processing Time'].values[0] <= s[j, stage] + M * (1 - seq[i, j, stage]))
                model.addConstr(s[j, stage] + data.loc[data['Job ID'] == j, f'Stage-{stage} Processing Time'].values[0] <= s[i, stage] + M * seq[i, j, stage])


# 5. Linearized tardiness calculation
for j in jobs:
    d = data.loc[data['Job ID'] == j, 'Due Time'].values[0]
    model.addConstr(T[j] >= c[j, 2] - d)
    model.addConstr(T[j] >= 0)

# Optimize the model
model.optimize()

if model.status == GRB.INFEASIBLE:
    print("Model is infeasible; computing IIS")
    model.computeIIS()
    model.write("model.ilp")

if model.status == GRB.OPTIMAL:
    print(f"Total Tardiness: {model.objVal}")
    # for j in jobs:
    #     print(f"Job {j} Tardiness: {T[j].X}")
    #     for stage in [1, 2]:
    #         print(f"Stage {stage} start time: {s[j, stage].X}, completion time: {c[j, stage].X}")


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[rosetta2] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 410 rows, 350 columns and 1170 nonzeros
Model fingerprint: 0xbb6dda5f
Variable types: 50 continuous, 300 integer (300 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e-01, 1e+04]
Presolve removed 32 rows and 142 columns
Presolve time: 0.00s
Presolved: 378 rows, 208 columns, 1116 nonzeros
Variable types: 28 continuous, 180 integer (180 binary)

Root relaxation: objective 2.400000e+00, 84 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.40000    0   28          -    2.40000      -     -    0s
H    0     0    